# OMERO Micro-SAM Workflow




## 1. Setup and Installation

In [1]:
# Import the streamlined package
import omero_annotate_ai
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_workflow_widget,
    create_pipeline
)

# System imports
import os
from pathlib import Path

print(f"📦 omero-annotate-ai version: {omero_annotate_ai.__version__}")
print(f"🔗 Available widgets: Connection, Workflow")
print(f"✨ Streamlined 2-widget workflow ready!")

# Check dependencies
try:
    import ezomero
    print(f"🔗 OMERO functionality: ✅ Available")
except ImportError:
    print(f"🔗 OMERO functionality: ❌ Install with: pip install -e .[omero]")

try:
    import keyring
    print(f"🔑 Keyring support: ✅ Available")
except ImportError:
    print(f"🔑 Keyring support: ⚠️ Not available (manual password entry only)")

📦 omero-annotate-ai version: 0.1.0
🔗 Available widgets: Connection, Workflow
✨ Streamlined 2-widget workflow ready!
🔗 OMERO functionality: ✅ Available
🔑 Keyring support: ✅ Available


## 2. OMERO Connection

In [ ]:
# Create and display the OMERO connection widget
print("🔌 OMERO Connection Setup")
print("Use the widget below to connect to your OMERO server:")
print("  • Fill in server details (host, username, password)")
print("  • Test connection before proceeding")
print("  • Choose password storage duration if desired")
print("  • Click 'Save & Connect' to establish connection")
print()

conn_widget = create_omero_connection_widget()
conn_widget.display()

print("\n📝 Next Step: Run the cell below after connecting to OMERO")

🔌 OMERO Connection Setup
Use the widget below to connect to your OMERO server:
  • Fill in server details (host, username, password)
  • Test connection before proceeding
  • Choose password storage duration if desired
  • Click 'Save & Connect' to establish connection

📄 Loaded configuration from connection history: root@localhost
🔐 Password loaded from keychain (no expiration)



📝 Next Step: Run the cell below after connecting to OMERO


In [3]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("❌ No OMERO connection established. Please use the widget above to connect.")

print("✅ OMERO connection established!")
print(f"👤 User: {conn.getUser().getName()}")
print(f"🏢 Group: {conn.getGroupFromContext().getName()}")
print(f"🔐 Secure: {conn.isSecure()}")
print(f"🔗 Connection ready for workflow setup")

✅ OMERO connection established!
👤 User: root
🏢 Group: system
🔐 Secure: True
🔗 Connection ready for workflow setup


## 3. Workflow Configuration

In [ ]:
# Create and display the workflow widget
print("🔬 OMERO Annotation Workflow Setup")
print("Follow the sequential workflow below:")
print("  1. Select working directory")
print("  2. Choose OMERO container")
print("  3. Check existing annotation tables")
print("  4. Configure micro-SAM parameters")
print("  5. Save configuration")
print()

workflow_widget = create_workflow_widget(connection=conn)
workflow_widget.display()

print("\n📝 Next Step: Complete the workflow above, then run the cell below")

🔬 OMERO Annotation Workflow Setup
Follow the sequential workflow below:
  1. Select working directory
  2. Choose OMERO container
  3. Check existing annotation tables
  4. Configure micro-SAM parameters
  5. Save configuration




📝 Next Step: Complete the workflow above, then run the cell below


## 4. Pipeline Execution


In [6]:
# Get configuration from workflow widget
config = workflow_widget.get_config()

# Display configuration summary
print("\n📋 Configuration Summary:")
print(f"   📦 Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"   🎯 Training Set: {config.name}")
print(f"   🔬 Model: {config.ai_model.model_type}")
print(f"   📺 Channel: {config.spatial_coverage.channels}")
print(f"   📁 Output: {config.output.output_directory}")
print(f"   🔄 Resume from Table: {config.workflow.resume_from_table}")
print(f"   📖 Read-only Mode: {config.workflow.read_only_mode}")

if config.processing.use_patches:
    print(f"   🧩 Patches: {config.processing.patches_per_image} per image ({config.processing.patch_size[0]}×{config.processing.patch_size[1]})"
    )

if config.spatial_coverage.three_d:
    print(f"   🧊 3D processing: Enabled")

print(f"\n📊 Processing scope: {'All images' if config.training.segment_all else f'{config.training.train_n} training + {config.training.validate_n} validation'}")


📋 Configuration Summary:
   📦 Container: project (ID: 201)
   🎯 Training Set: micro_sam_annotation
   🔬 Model: vit_b_lm
   📺 Channel: [0]
   📁 Output: c:\Users\Maarten\Documents\Github\omero_annotate_ai\examples\omero_annotations
   🔄 Resume from Table: False
   📖 Read-only Mode: False

📊 Processing scope: 3 training + 3 validation


In [7]:
# Create pipeline and preview what will be processed
pipeline = create_pipeline(config, conn)

# Get container details
container_type = config.omero.container_type
container_id = config.omero.container_id

print(f"🔍 Validating {container_type} with ID {container_id}...")

# Validate container exists
container = conn.getObject(container_type.capitalize(), container_id)
if container is None:
    raise ValueError(f"{container_type.capitalize()} with ID {container_id} not found")

print(f"✅ Found {container_type}: {container.getName()}")
if container.getDescription():
    print(f"📝 Description: {container.getDescription()}")

# Get list of images that will be processed
try:
    images_list = pipeline.get_images_from_container()
    print(f"\n📊 Found {len(images_list)} images to process")
    
    # Show sample images
    print("\n🖼️ Sample images:")
    for i, img in enumerate(images_list[:5]):
        if hasattr(img, 'getName'):
            print(f"   {i+1}. {img.getName()} (ID: {img.getId()})")
        else:
            img_obj = conn.getObject("Image", img)
            if img_obj:
                print(f"   {i+1}. {img_obj.getName()} (ID: {img})")
    
    if len(images_list) > 5:
        print(f"   ... and {len(images_list) - 5} more images")
        
except Exception as e:
    print(f"❌ Error getting images from container: {e}")
    raise

print(f"\n✅ Ready to process {len(images_list)} images!")

🔍 Validating project with ID 201...
✅ Found project: foci_test
Loading images from project 201
Found 10 images

📊 Found 10 images to process

🖼️ Sample images:
   1. 001_MAX_ExpMP2410_005_IB10#1_2h, Position001.tif (ID: 451)
   2. 002_MAX_ExpMP2410_005_IB10#1_2h, Position002.tif (ID: 452)
   3. 009_MAX_ExpMP2410_005_IB10#1_noIR, Position001.tif (ID: 453)
   4. 010_MAX_ExpMP2410_005_IB10#1_noIR, Position002.tif (ID: 454)
   5. 017_MAX_ExpMP2410_005_D3_noIR, Position001.tif (ID: 455)
   ... and 5 more images

✅ Ready to process 10 images!


In [8]:
# Run the complete annotation pipeline
print("🚀 Starting annotation pipeline...")
print(f"   📊 Processing {len(images_list)} images")
print(f"   🔬 Using micro-SAM model: {config.ai_model.model_type}")

if config.processing.batch_size == 0:
    print(f"   ⚡ Processing: All images in one batch")
else:
    print(f"   📦 Processing: Batches of {config.processing.batch_size} images")

print(f"   🎨 Napari will open for interactive annotation")
print(f"   📝 Close napari windows when annotation is complete")
print()

try:
    # Run the complete workflow - this is the key call!
    table_id, processed_images = pipeline.run_full_workflow()
    
    print(f"\n🎉 Annotation pipeline completed successfully!")
    print(f"📊 Processed {len(processed_images)} images")
    print(f"📋 Tracking table ID: {table_id}")
    
    if config.workflow.read_only_mode:
        print(f"💾 Annotations saved locally to: {config.output.output_directory}")
    else:
        print(f"☁️ Annotations uploaded to OMERO")
        
except Exception as e:
    print(f"❌ Error during annotation pipeline: {e}")
    import traceback
    traceback.print_exc()
    raise

🚀 Starting annotation pipeline...
   📊 Processing 10 images
   🔬 Using micro-SAM model: vit_b_lm
   ⚡ Processing: All images in one batch
   🎨 Napari will open for interactive annotation
   📝 Close napari windows when annotation is complete

Creating annotation table
Loading images from project 201
Found 10 images
Creating table for 10 images with model: vit_b_lm


INFO:omero_annotate_ai.core.pipeline:Creating new tracking table: micro_sam_training_micro_sam_annotation
INFO:omero_annotate_ai.core.pipeline:ROI namespace: omero_annotate_ai.table.micro_sam_training_micro_sam_annotation
c:\Users\Maarten\miniforge3\envs\micro-sam\Lib\site-packages\omero_annotate_ai\omero\omero_functions.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(False).astype(bool)
c:\Users\Maarten\miniforge3\envs\micro-sam\Lib\site-packages\omero_annotate_ai\omero\omero_functions.py:135: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downca

📋 Created tracking table 'micro_sam_training_micro_sam_annotation' with 6 units
   Container: project 201
   Table ID: 1331
object group 0
Stored configuration as annotation ID: 1332
object group 0
📊 Workflow status updated: 0/6 (0.0%) - pending
Created annotation table with ID: 1331
Starting annotation processing from table ID: 1331
[TABLE] Getting unprocessed units from table 1331
📋 Found 6 unprocessed units
Found 6 processing units
Processing batch 1/1
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy...
   Processing chunk 1/1


INFO:omero.gateway:Registered cf2819a3-a06e-497b-b867-954f914504e3/6bbcb269-5c50-437d-818e-0c3853f73aa4omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 43859 -t 60000
INFO:omero.gateway:Unregistered cf2819a3-a06e-497b-b867-954f914504e3/6bbcb269-5c50-437d-818e-0c3853f73aa4omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 43859 -t 60000
INFO:omero.gateway:Registered cf2819a3-a06e-497b-b867-954f914504e3/0527ea99-3231-4342-a011-8bfcb041b028omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 43859 -t 60000


✅ Successfully loaded 1 images
📊 Loading 1 images using dask...
💾 Materializing dask arrays to numpy...
   Processing chunk 1/1


...

INFO:omero.gateway:Registered cf2819a3-a06e-497b-b867-954f914504e3/ee8fc71a-55e6-4168-8998-09d947f17fedomero.api.RawFileStore -t -e 1.1:tcp -h 172.19.0.8 -p 43859 -t 60000


🏷️ Found 36 unique labels: [0 1 2 3 4 5 6 7 8 9]...
🔍 Step 2: Converting labels to ROI shapes...
✅ Created 35 ROI shapes from labels
🔍 Step 3: Uploading label file as attachment


INFO:omero.gateway:Unregistered cf2819a3-a06e-497b-b867-954f914504e3/ee8fc71a-55e6-4168-8998-09d947f17fedomero.api.RawFileStore -t -e 1.1:tcp -h 172.19.0.8 -p 43859 -t 60000


✅ File annotation uploaded with ID: 1339
🔍 Step 4: Uploading ROI shapes
✅ Created 35 ROI shapes for image 454 with ID: 319
☁️ Uploaded annotations from c:\Users\Maarten\Documents\Github\omero_annotate_ai\examples\omero_annotations\annotations\seg_00005.tif to image 454
   File annotation ID: 1339
   ROI ID: 319
Error processing batch 1: 'TrainingConfig' object has no attribute 'annotation_type'
Annotation processing failed - no units were processed

🎉 Annotation pipeline completed successfully!
📊 Processed 10 images
📋 Tracking table ID: 1331
☁️ Annotations uploaded to OMERO


Traceback (most recent call last):
  File "c:\Users\Maarten\miniforge3\envs\micro-sam\Lib\site-packages\omero_annotate_ai\core\pipeline.py", line 674, in run_annotation
    table_id = self._process_annotation_results(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Maarten\miniforge3\envs\micro-sam\Lib\site-packages\omero_annotate_ai\core\pipeline.py", line 538, in _process_annotation_results
    annotation_type=self.config.training.annotation_type,
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Maarten\miniforge3\envs\micro-sam\Lib\site-packages\pydantic\main.py", line 892, in __getattr__
    raise AttributeError(f'{type(self).__name__!r} object has no attribute {item!r}')
AttributeError: 'TrainingConfig' object has no attribute 'annotation_type'


## 5. Results and Export

Review the results and export your configuration for future use.

In [14]:
# Display results summary
print("📊 Pipeline Results Summary:")
print(f"   🎯 Training Set: {config.name}")
print(f"   📋 Tracking Table ID: {table_id}")
print(f"   📊 Images Processed: {len(processed_images)}")
print(f"   📦 Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"   🔬 Model Used: {config.ai_model.model_type}")
print(f"   📁 Output Location: {config.output.output_directory}")

# Show processed images
if processed_images:
    print(f"\n🖼️ Processed Images:")
    for i, img_obj in enumerate(processed_images[:10]):
        if img_obj:
            print(f"   {i + 1}. {img_obj.getName()} (ID: {img_obj.getId()})")

    if len(processed_images) > 10:
        print(f"   ... and {len(processed_images) - 10} more images")

print(f"\n✅ Pipeline completed successfully!")

📊 Pipeline Results Summary:
   🎯 Training Set: micro_sam_annotation
   📋 Tracking Table ID: 1331
   📊 Images Processed: 10
   📦 Container: project (ID: 201)
   🔬 Model Used: vit_b_lm
   📁 Output Location: c:\Users\Maarten\Documents\Github\omero_annotate_ai\examples\omero_annotations

🖼️ Processed Images:
   1. 001_MAX_ExpMP2410_005_IB10#1_2h, Position001.tif (ID: 451)
   2. 002_MAX_ExpMP2410_005_IB10#1_2h, Position002.tif (ID: 452)
   3. 009_MAX_ExpMP2410_005_IB10#1_noIR, Position001.tif (ID: 453)
   4. 010_MAX_ExpMP2410_005_IB10#1_noIR, Position002.tif (ID: 454)
   5. 017_MAX_ExpMP2410_005_D3_noIR, Position001.tif (ID: 455)
   6. 018_MAX_ExpMP2410_005_D3_noIR, Position002.tif (ID: 456)
   7. 025_MAX_ExpMP2410_005_D3_2h, Position001.tif (ID: 457)
   8. 026_MAX_ExpMP2410_005_D3_2h, Position002.tif (ID: 458)
   9. 041_MAX_ExpMP2410_005_A2_noIR, Position001.tif (ID: 459)
   10. 042_MAX_ExpMP2410_005_A2_noIR, Position002.tif (ID: 460)

✅ Pipeline completed successfully!


In [18]:
# Export configuration for future use
config_filename = f"annotation_config_{config.name}.yaml"
config_path = Path(config.output.output_directory) / config_filename

try:
    config.save_yaml(config_path)
    print(f"💾 Configuration saved to: {config_path}")
except Exception as e:
    print(f"⚠️ Could not save to output folder, saving to current directory")
    config.save_yaml(config_filename)
    print(f"💾 Configuration saved to: {config_filename}")

print(f"\n📋 Configuration Summary:")
print(f"   Name: {config.name}")
print(f"   Description: {config.description}")
print(f"   Output Directory: {config.output.output_directory}")
print(f"   Model Type: {config.ai_model.model_type}")

print(f"\n🔄 To reuse this configuration:")
print(f"```python")
print(f"from omero_annotate_ai import load_config")
print(f"config = load_config('{config_filename}')")
print(f"```")

💾 Configuration saved to: c:\Users\Maarten\Documents\Github\omero_annotate_ai\examples\omero_annotations\annotation_config_micro_sam_annotation.yaml

📋 Configuration Summary:
   Name: micro_sam_annotation


AttributeError: 'AnnotationConfig' object has no attribute 'description'

## 6. Cleanup

Close the OMERO connection when finished.

In [ ]:
# Close OMERO connection
if 'conn' in locals() and conn is not None:
    conn.close()
    print("🔌 OMERO connection closed")

print(f"\n🎉 Streamlined workflow completed!")
print(f"📊 Total images processed: {len(processed_images) if 'processed_images' in locals() else 0}")
print(f"💾 Configuration saved for future use")
print(f"✨ Ready for next annotation workflow!")